In [1]:
import pandas as pd

file_path = r"D:\News.csv"
data = pd.read_csv(file_path)
data.head()

,ID,News Category,Title,Summary
0,N88753,lifestyle,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N45436,news,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...
2,N23144,health,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
3,N86255,health,Dispose of unwanted prescription drugs during ...,NaN
4,N93187,news,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...


In [2]:
categories = data["News Category"].value_counts()

In [3]:
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

feature = data["Title"].tolist()
tfidf = text.TfidfVectorizer(stop_words="english")

In [4]:
tfidf_matrix = tfidf.fit_transform(feature)
similarity = cosine_similarity(tfidf_matrix)

In [5]:
indices = pd.Series(data.index, index=data['Title']).drop_duplicates()

In [6]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\srihitha
[nltk_data]     pulapa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
data['sentiment_score'] = data['Title'].apply(lambda x: sia.polarity_scores(x)['compound'])

def news_recommendation(Title):
    index = indices[Title]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = [(i, score) for i, score in similarity_scores if data.iloc[i]['sentiment_score'] > 0]
    similarity_scores = [(i, score * data.iloc[i]['sentiment_score']) for i, score in similarity_scores]
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    newsindices = [i[0] for i in similarity_scores]
    return data['Title'].iloc[newsindices]

print(news_recommendation("Walmart Slashes Prices on Last-Generation iPads"))

83827    Walmart's Black Friday 2019 ad: the best deals...
24600    Best Amazon deals: Get incredible prices on po...
89439    Don't believe the hype, car enthusiasm is safe...
87613    Bay Area colleges celebrate first-generation s...
78383    How to Get the Best Walmart Black Friday 2019 ...
12191    There Are Some Great Finds in Walmart's Early ...
92529    The Best Gifts to Give from Walmart at Any Pri...
89688    Brave browser 1.0 arrives, bringing its privac...
5915     The best Apple Watch 4 prices and deals in Oct...
68758    America's favorite cookbook is this family's l...
Name: Title, dtype: object
